# KITTI Dataset augmentation visualization

## Dataset structure

```
📦gta
┗ 📂training
  ┣ 📂calib
  ┃ ┣ 📜<file_id>.txt
  ┃ ┗ 📜 ...
  ┣ 📂image_2
  ┃ ┣ 📜<file_id>.png
  ┃ ┗ 📜 ...
  ┣ 📂label_2
  ┃ ┣ 📜<file_id>.txt
  ┃ ┗ 📜 ...
  ┗ 📂velodyne
    ┣ 📜<file_id>.bin
    ┗ 📜 ...
```

In [1]:
import os
import mayavi.mlab as mlab
import numpy as np
from copy import deepcopy
import pickle

import pcdet.datasets.augmentor

from pcdet.config import cfg, cfg_from_yaml_file
from pcdet.datasets import build_dataloader
from pcdet.utils import common_utils
from pcdet.models import build_network
from tools.visual_utils import visualize_utils as V

mlab.init_notebook()

Notebook initialized with ipy backend.


In [2]:
# generate pickles from the kitti data
# os.system("python -m pcdet.datasets.kitti.kitti_dataset create_kitti_infos tools/cfgs/dataset_configs/kitti_dataset.yaml")

## Visualizing the LiDAR point cloud with labels

In [3]:
logger = common_utils.create_logger()

In [4]:
cfg_from_yaml_file('tools/cfgs/dataset_configs/kitti_dataset.yaml', cfg)
# cfg_from_yaml_file('tools/cfgs/kitti_models/pointpillar_augs.yaml', cfg)

cfg.DATA_PATH = 'data/kitti'

train_set, train_loader, train_sampler = build_dataloader(
    dataset_cfg=cfg,
    class_names=['Car', 'Pedestrian', 'Cyclist'],
    batch_size=1,
    dist=False,
    workers=4,
    logger=logger,
    training=True,
    merge_all_iters_to_one_epoch=False,
    total_epochs=0
)

logger.info(f'Total number of samples: \t{len(train_set)}')

data_dict_list = []
logger.info('Loading samples')
for idx, data_dict in enumerate(train_set):
    logger.info(f'Loaded sample index: \t{idx + 1}')
    data_dict = train_set.collate_batch([data_dict])
    data_dict_list.append(data_dict)
    if idx > 4: break


2021-04-29 11:26:50,292   INFO  Database filter by min points Car: 189 => 177
2021-04-29 11:26:50,293   INFO  Database filter by min points Pedestrian: 25 => 25
2021-04-29 11:26:50,293   INFO  Database filter by min points Cyclist: 10 => 10
2021-04-29 11:26:50,294   INFO  Database filter by difficulty Car: 177 => 137
2021-04-29 11:26:50,294   INFO  Database filter by difficulty Pedestrian: 25 => 24
2021-04-29 11:26:50,295   INFO  Database filter by difficulty Cyclist: 10 => 9
2021-04-29 11:26:50,353   INFO  Loading KITTI dataset
2021-04-29 11:26:50,356   INFO  Total samples for KITTI dataset: 58
2021-04-29 11:26:50,357   INFO  Total number of samples: 	58
2021-04-29 11:26:50,358   INFO  Loading samples
2021-04-29 11:26:50,590   INFO  Loaded sample index: 	1
2021-04-29 11:26:50,766   INFO  Loaded sample index: 	2
2021-04-29 11:26:51,009   INFO  Loaded sample index: 	3
2021-04-29 11:26:51,174   INFO  Loaded sample index: 	4
2021-04-29 11:26:51,349   INFO  Loaded sample index: 	5
2021-04-

In [5]:
cfg_from_yaml_file('tools/cfgs/kitti_models/pointpillar_augs.yaml', cfg)
;

''

In [6]:
model = build_network(model_cfg=cfg.MODEL, num_class=len(cfg.CLASS_NAMES), dataset=train_set)

RuntimeError: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero.

In [ ]:
batch_dict = data_dict_list[2]
pred_dicts = None
#annos = dataset.generate_prediction_dicts(batch_dict, pred_dicts, ['Car', 'Pedestrian', 'Cyclist'])

In [ ]:
info = train_set.get_infos()
;

In [ ]:
info[2]

In [ ]:
data_dict_list

In [ ]:
with open('data/kitti/kitti_infos_train.pkl', "rb") as pickle_file:
    info = pickle.load(pickle_file)

In [ ]:
scene = data_dict_list[2]

In [ ]:
scene

In [ ]:
def show_pc(data_dict):
    V.draw_scenes(
        points=data_dict['points'][:, 1:, gt_boxes=data_dict['gt_boxes'][0]
    )

    return mlab.test_plot3d()


## Visualizing the original scene
![Original Camera Image](data/kitti/training/image_2/000004.png)


In [ ]:
show_pc(scene)
#mlab.savefig('figs/aug_pipeline_output.png')
#mlab.test_plot3d()

In [ ]:
for scene in data_dict_list:
    label_text = ""
    for idx, name in enumerate(scene['gt_names']):
        label_text = label_text+'%s %.2f %d %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f %.2f'
    % (name, scene['truncation'][idx], scene['occlusionA'][idx], scene['alpha'][idx], bbox[idx][0], bbox[idx][1], bbox[idx][2], bbox[idx][3],
    dims[idx][1], dims[idx][2], dims[idx][0], loc[idx][0], loc[idx][1], loc[idx][2],
    single_pred_dict['rotation_y'][idx], single_pred_dict['score'][idx])
    with open(f'data/augmented_kitti/training/label_2/{}.txt', 'w') as label_file:
        

## Augmentation methods

### Random flip

In [ ]:
def random_flip_along_x(gt_boxes, points):
    """
    Args:
        gt_boxes: (N, 7 + C), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C)
    Returns:
    """
    enable = True
    if enable:
        gt_boxes[:, 1] = -gt_boxes[:, 1]
        gt_boxes[:, 6] = -gt_boxes[:, 6]
        points[:, 1] = -points[:, 1]

        if gt_boxes.shape[1] > 8:
            gt_boxes[:, 8] = -gt_boxes[:, 8]

    return gt_boxes, points

def random_flip_along_y(gt_boxes, points):
    """
    Args:
        gt_boxes: (N, 7 + C), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C)
    Returns:
    """
    enable = True
    if enable:
        gt_boxes[:, 0] = -gt_boxes[:, 0]
        gt_boxes[:, 6] = -(gt_boxes[:, 6] + np.pi)
        points[:, 0] = -points[:, 0]

        if gt_boxes.shape[1] > 7:
            gt_boxes[:, 7] = -gt_boxes[:, 7]

    return gt_boxes, points

In [ ]:
%%time
flipped_y = deepcopy(scene)
flipped_y['gt_boxes'][0], flipped_y['points'][:,1:] = random_flip_along_y(flipped_y['gt_boxes'][0], flipped_y['points'][:,1:])

In [ ]:
show_pc(flipped_y)
mlab.savefig('figs/flipped_y.png')
mlab.test_plot3d()

In [ ]:
%%time
flipped_x = deepcopy(scene)
flipped_x['gt_boxes'][0], flipped_x['points'][:,1:] = random_flip_along_x(flipped_x['gt_boxes'][0], flipped_x['points'][:,1:])

In [ ]:
show_pc(flipped_x)
mlab.savefig('figs/flipped_x.png')
mlab.test_plot3d()

### Global rotation

In [ ]:
def global_rotation(gt_boxes, points, rot_range):
    """
    Args:
        gt_boxes: (N, 7 + C), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        rot_range: [min, max]
    Returns:
    """
    noise_rotation = np.random.uniform(rot_range[0], rot_range[1])
    points = common_utils.rotate_points_along_z(points[np.newaxis, :, :], np.array([noise_rotation]))[0]
    gt_boxes[:, 0:3] = common_utils.rotate_points_along_z(gt_boxes[np.newaxis, :, 0:3], np.array([noise_rotation]))[0]
    gt_boxes[:, 6] += noise_rotation
    if gt_boxes.shape[1] > 8:
        gt_boxes[:, 7:9] = common_utils.rotate_points_along_z(
            np.hstack((gt_boxes[:, 7:9], np.zeros((gt_boxes.shape[0], 1))))[np.newaxis, :, :],
            np.array([noise_rotation])
        )[0][:, 0:2]

    return gt_boxes, points

In [ ]:
%%time
rotated = deepcopy(scene)
rotated['gt_boxes'][0], rotated['points'][:,1:] = global_rotation(rotated['gt_boxes'][0], rotated['points'][:,1:], [-0.78539816, 0.78539816])

In [ ]:
show_pc(rotated)
mlab.savefig('figs/rotated_global.png')
mlab.test_plot3d()

### Global scaling

In [ ]:
def global_scaling(gt_boxes, points, scale_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading]
        points: (M, 3 + C),
        scale_range: [min, max]
    Returns:
    """
    if scale_range[1] - scale_range[0] < 1e-3:
        return gt_boxes, points
    noise_scale = np.random.uniform(scale_range[0], scale_range[1])
    points[:, :3] *= noise_scale
    gt_boxes[:, :6] *= noise_scale
    return gt_boxes, points

In [ ]:
%%time
scaled = deepcopy(scene)
scaled['gt_boxes'][0], scaled['points'][:,1:] = global_scaling(scaled['gt_boxes'][0], scaled['points'][:, 1:], [0.2, 0.5])

In [ ]:
show_pc(scaled)
mlab.savefig('figs/scaled_global.png')
mlab.test_plot3d()

### Global translation

In [ ]:
def random_translation_along_x(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    offset = np.random.uniform(offset_range[0], offset_range[1])

    points[:, 0] += offset
    gt_boxes[:, 0] += offset

    if gt_boxes.shape[1] > 7:
        gt_boxes[:, 7] += offset

    return gt_boxes, points

def random_translation_along_y(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    offset = np.random.uniform(offset_range[0], offset_range[1])

    points[:, 1] += offset
    gt_boxes[:, 1] += offset

    if gt_boxes.shape[1] > 8:
        gt_boxes[:, 8] += offset

    return gt_boxes, points

def random_translation_along_z(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    offset = np.random.uniform(offset_range[0], offset_range[1])

    points[:, 2] += offset
    gt_boxes[:, 2] += offset

    return gt_boxes, points

In [ ]:
%%time
translation_x = deepcopy(scene)
translation_x['gt_boxes'][0], translation_x['points'][:,1:] = random_translation_along_x(translation_x['gt_boxes'][0], translation_x['points'][:, 1:], [15, 20])

In [ ]:
show_pc(translation_x)
mlab.savefig('figs/translation_global_x.png')
mlab.test_plot3d()

In [ ]:
%%time
translation_y = deepcopy(scene)
translation_y['gt_boxes'][0], translation_y['points'][:,1:] = random_translation_along_y(translation_y['gt_boxes'][0], translation_y['points'][:, 1:], [15, 20])

In [ ]:
show_pc(translation_y)
mlab.savefig('figs/translation_global_y.png')
mlab.test_plot3d()

In [ ]:
%%time
translation_z = deepcopy(scene)
translation_z['gt_boxes'][0], translation_z['points'][:,1:] = random_translation_along_z(translation_z['gt_boxes'][0], translation_z['points'][:, 1:], [15, 20])

In [ ]:
show_pc(translation_z)
mlab.savefig('figs/translation_global_z.png')
mlab.test_plot3d()

### Global frustum dropout

In [ ]:
def global_frustum_dropout_top(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.max(points[:, 2]) - intensity * (np.max(points[:, 2]) - np.min(points[:, 2]))
    points = points[points[:,2] < threshold]
    gt_boxes = gt_boxes[gt_boxes[:,2] < threshold]

    return gt_boxes, points

def global_frustum_dropout_bottom(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.min(points[:, 2]) + intensity * (np.max(points[:, 2]) - np.min(points[:, 2]))
    points = points[points[:,2] > threshold]
    gt_boxes = gt_boxes[gt_boxes[:,2] > threshold]

    return gt_boxes, points

def global_frustum_dropout_left(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.max(points[:, 1]) - intensity * (np.max(points[:, 1]) - np.min(points[:, 1]))
    points = points[points[:,1] < threshold]
    gt_boxes = gt_boxes[gt_boxes[:,1] < threshold]

    return gt_boxes, points

def global_frustum_dropout_right(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    intensity = np.random.uniform(intensity_range[0], intensity_range[1])

    threshold = np.min(points[:, 1]) + intensity * (np.max(points[:, 1]) - np.min(points[:, 1]))
    points = points[points[:,1] > threshold]
    gt_boxes = gt_boxes[gt_boxes[:,1] > threshold]

    return gt_boxes, points

In [ ]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_top(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

In [ ]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_top.png')
mlab.test_plot3d()

In [ ]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_bottom(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

In [ ]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_bottom.png')
mlab.test_plot3d()

In [ ]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_left(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

In [ ]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_left.png')
mlab.test_plot3d()

In [ ]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = global_frustum_dropout_right(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.4, 0.5])

In [ ]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/frustum_dropout_right.png')
mlab.test_plot3d()

#### Aux function to find which points are inside a bounding box

In [ ]:
def get_points_in_box(points, gt_box):
    x, y, z, dx, dy, dz = gt_box[0], gt_box[1], gt_box[2], gt_box[3], gt_box[4], gt_box[5]

    mask = np.logical_and(points[:,0] <= x + dx/2, \
         np.logical_and(points[:,0] >= x - dx/2, \
             np.logical_and(points[:,1] <= y + dy/2, \
                 np.logical_and(points[:,1] >= y - dy/2, \
                     np.logical_and(points[:,2] <= z + dz/2, points[:,2] >= z - dz/2)))))

    points = points[mask]

    return points, mask

In [ ]:
%%time
points_to_filter = deepcopy(scene)
filtered_points, mask = get_points_in_box(points_to_filter['points'][:,1:], points_to_filter['gt_boxes'][0][1])

In [ ]:
V.draw_scenes(
        points=filtered_points, gt_boxes=points_to_filter['gt_boxes'][0,1:]
    )

mlab.savefig('figs/points_in_box.png')
mlab.test_plot3d()

### Local translation

In [ ]:
def local_translation_along_x(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        offset = np.random.uniform(offset_range[0], offset_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        points[mask, 0] += offset

        gt_boxes[idx, 0] += offset

        if gt_boxes.shape[1] > 7:
            gt_boxes[idx, 7] += offset

    return gt_boxes, points


def local_translation_along_y(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        offset = np.random.uniform(offset_range[0], offset_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        points[mask, 1] += offset

        gt_boxes[idx, 1] += offset

        if gt_boxes.shape[1] > 8:
            gt_boxes[idx, 8] += offset

    return gt_boxes, points


def local_translation_along_z(gt_boxes, points, offset_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        offset_range: [min max]]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        offset = np.random.uniform(offset_range[0], offset_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        points[mask, 2] += offset

        gt_boxes[idx, 2] += offset

    return gt_boxes, points

In [ ]:
%%time
local_translation_x = deepcopy(scene)
local_translation_x['gt_boxes'][0], local_translation_x['points'][:,1:] = local_translation_along_x(local_translation_x['gt_boxes'][0], local_translation_x['points'][:, 1:], [10, 15])

In [ ]:
show_pc(local_translation_x)
mlab.savefig('figs/translation_local_x.png')
mlab.test_plot3d()

In [ ]:
%%time
local_translation_y = deepcopy(scene)
local_translation_y['gt_boxes'][0], local_translation_y['points'][:,1:] = local_translation_along_y(local_translation_y['gt_boxes'][0], local_translation_y['points'][:, 1:], [10, 15])

In [ ]:
show_pc(local_translation_y)
mlab.savefig('figs/translation_local_y.png')
mlab.test_plot3d()

In [ ]:
%%time
local_translation_z = deepcopy(scene)
local_translation_z['gt_boxes'][0], local_translation_z['points'][:,1:] = local_translation_along_z(local_translation_z['gt_boxes'][0], local_translation_z['points'][:, 1:], [10, 15])

In [ ]:
show_pc(local_translation_z)
mlab.savefig('figs/translation_local_z.png')
mlab.test_plot3d()

### Local scaling

In [ ]:
def local_scaling(gt_boxes, points, scale_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading]
        points: (M, 3 + C),
        scale_range: [min, max]
    Returns:
    """
    if scale_range[1] - scale_range[0] < 1e-3:
        return gt_boxes, points

    for idx, box in enumerate(gt_boxes):
        noise_scale = np.random.uniform(scale_range[0], scale_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        
        # tranlation to axis center
        points[mask, 0] -= box[0]
        points[mask, 1] -= box[1]
        points[mask, 2] -= box[2]

        # apply scaling
        points[mask, :3] *= noise_scale

        # tranlation back to original position
        points[mask, 0] += box[0]
        points[mask, 1] += box[1]
        points[mask, 2] += box[2]

        gt_boxes[idx, 3:6] *= noise_scale
    return gt_boxes, points

In [ ]:
%%time
scaled_local = deepcopy(scene)
scaled_local['gt_boxes'][0], scaled_local['points'][:,1:] = local_scaling(scaled_local['gt_boxes'][0], scaled_local['points'][:, 1:], [0.2, 0.5])

In [ ]:
show_pc(scaled_local)
mlab.savefig('figs/scaled_local.png')
mlab.test_plot3d()

### Local rotation

In [ ]:
def local_rotation(gt_boxes, points, rot_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]]
        points: (M, 3 + C),
        rot_range: [min, max]
    Returns:
    """
    
    for idx, box in enumerate(gt_boxes):
        noise_rotation = np.random.uniform(rot_range[0], rot_range[1])
        points_in_box, mask = get_points_in_box(points, box)
        
        centroid_x = box[0]
        centroid_y = box[1]
        centroid_z = box[2]

        # tranlation to axis center
        points[mask, 0] -= centroid_x
        points[mask, 1] -= centroid_y
        points[mask, 2] -= centroid_z
        box[0] -= centroid_x
        box[1] -= centroid_y
        box[2] -= centroid_z

        # apply rotation
        points[mask, :] = common_utils.rotate_points_along_z(points[np.newaxis, mask, :], np.array([noise_rotation]))[0]
        box[0:3] = common_utils.rotate_points_along_z(box[np.newaxis, np.newaxis, 0:3], np.array([noise_rotation]))[0][0]

        # tranlation back to original position
        points[mask, 0] += centroid_x
        points[mask, 1] += centroid_y
        points[mask, 2] += centroid_z
        box[0] += centroid_x
        box[1] += centroid_y
        box[2] += centroid_z

        gt_boxes[idx, 6] += noise_rotation
        if gt_boxes.shape[1] > 8:
            gt_boxes[idx, 7:9] = common_utils.rotate_points_along_z(
                np.hstack((gt_boxes[idx, 7:9], np.zeros((gt_boxes.shape[0], 1))))[np.newaxis, :, :],
                np.array([noise_rotation])
            )[0][:, 0:2]

    return gt_boxes, points

In [ ]:
%%time
rotated_local = deepcopy(scene)
rotated_local['gt_boxes'][0], rotated_local['points'][:,1:] = local_rotation(rotated_local['gt_boxes'][0],rotated_local['points'][:, 1:], [-0.78539815, 0.78539816])

In [ ]:
show_pc(rotated_local)
mlab.savefig('figs/rotated_local.png')
mlab.test_plot3d()

### Local frustum dropout

In [ ]:
def local_frustum_dropout_top(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box = get_points_in_box(points, box)
        threshold = (z + dz/2) - intensity * dz

        points = points[np.logical_not( \
        np.logical_and(points[:,1] <= y + dy/2, \
        np.logical_and(points[:,1] >= y - dy/2, \
            np.logical_and(points[:,0] <= x + dx/2, \
                np.logical_and(points[:,0] >= x - dx/2, \
                    np.logical_and(points[:,2] <= z + dz/2, points[:,2] >= threshold))))))]

    return gt_boxes, points

def local_frustum_dropout_bottom(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box = get_points_in_box(points, box)
        threshold = (z - dz/2) + intensity * dz
        points = points[np.logical_not( \
            np.logical_and(points[:,1] <= y + dy/2, \
            np.logical_and(points[:,1] >= y - dy/2, \
                np.logical_and(points[:,0] <= x + dx/2, \
                    np.logical_and(points[:,0] >= x - dx/2, \
                        np.logical_and(points[:,2] <= threshold, points[:,2] >= z - dz/2))))))]

    return gt_boxes, points

def local_frustum_dropout_left(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box = get_points_in_box(points, box)
        threshold = (y + dy/2) - intensity * dy

        points = points[np.logical_not( \
            np.logical_and(points[:,1] <= y + dy/2, \
            np.logical_and(points[:,1] >= threshold, \
                np.logical_and(points[:,0] <= x + dx/2, \
                    np.logical_and(points[:,0] >= x - dx/2, \
                        np.logical_and(points[:,2] <= z + dz/2, points[:,2] >= z - dz/2))))))]

    return gt_boxes, points

def local_frustum_dropout_right(gt_boxes, points, intensity_range):
    """
    Args:
        gt_boxes: (N, 7), [x, y, z, dx, dy, dz, heading, [vx], [vy]],
        points: (M, 3 + C),
        intensity: [min, max]
    Returns:
    """
    for idx, box in enumerate(gt_boxes):
        x, y, z, dx, dy, dz = box[0], box[1], box[2], box[3], box[4], box[5]

        intensity = np.random.uniform(intensity_range[0], intensity_range[1])
        points_in_box = get_points_in_box(points, box)
        threshold = (y - dy/2) + intensity * dy

        points = points[np.logical_not( \
            np.logical_and(points[:,1] <= threshold, \
            np.logical_and(points[:,1] >= y - dy/2, \
                np.logical_and(points[:,0] <= x + dx/2, \
                    np.logical_and(points[:,0] >= x - dx/2, \
                        np.logical_and(points[:,2] <= z + dz/2, points[:,2] >= z - dz/2))))))]

    return gt_boxes, points

In [ ]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_top(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.5, 0.5])

In [ ]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_top.png')
mlab.test_plot3d()

In [ ]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_bottom(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.3, 0.4])

In [ ]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_bottom.png')
mlab.test_plot3d()

In [ ]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_left(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.3, 0.4])

In [ ]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_left.png')
mlab.test_plot3d()

In [ ]:
%%time
frustum_dropout = deepcopy(scene)
frustum_dropout['gt_boxes'], frustum_dropout['points'] = local_frustum_dropout_right(frustum_dropout['gt_boxes'][0], frustum_dropout['points'][:, 1:], [0.3, 0.4])

In [ ]:
V.draw_scenes(
        points=frustum_dropout['points'], gt_boxes=frustum_dropout['gt_boxes']
    )

mlab.savefig('figs/local_frustum_dropout_right.png')
mlab.test_plot3d()

In [ ]:
def saveKittiVelodyneFile(tuple_list, filename, directory):
    '''
    Saves pointcloud in binary file and is independent of the number of properties in the pointcloud points
    (x, y, z, reflectance)
    '''
    with open(directory + filename, "wb") as f:
        for point in tuple_list:
            s = struct.pack('f'*len(point), *point)
            f.write(s)

In [ ]:
def saveLabelInfo(self, dirname, filename):        
    '''
    (1 value) label_name: 'Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc' or 'DontCare'
    (1 value) trucanted: [0, 1], evaluates how much an object  isn't within the image boundaries
    (1 value) occluded: {0, 1, 2, 3}, fully-visible, partly occluded, largely-occuled, unknown
    (1 value) alpha: [-pi, pi], observation angle of the object
    (4 values) bbox: {left, top, right, bottom}, 2d bounding box of the object in the image
    (3 values) dimensions: {height, width, length}, 3d object dimensions in meters
    (3 values) location: {x, y, z}, 3d object location in camera coordinates in meters
    (1 value) rotation_y: [-pi, pi], rotation around Y-axis (up axis) in camera coordinates
    '''

def saveListIntoTxtFile(self, list_of_str, dirname, filename):
        '''
        Store list of strings into a file
        '''
        with open(os.path.join(dirname, filename), "w") as the_file:
            for i in range(0, len(list_of_str)):
                the_file.write(list_of_str[i] + "\n")